<a href="https://colab.research.google.com/github/agungfirdaus717-ux/torentotgd/blob/main/SubTranslator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ========================================
# 🔥 Google Colab - Multi SRT Translator
#    pakai deep-translator (Google Translate)
#    Tanpa API Key, auto-retry + proteksi tag
# ========================================

!pip install deep-translator tqdm

import os, re, time, random
from deep_translator import GoogleTranslator
from tqdm import tqdm
from google.colab import files

translator = GoogleTranslator(source="auto", target="id")

# =======================
# Proteksi tag/tagar
# =======================
def protect_text(text):
    """
    Ganti tag/simbol khusus jadi placeholder
    """
    placeholders = {}
    # <i>…</i>
    text, n = re.subn(r"<i>.*?</i>", lambda m: f"__TAG{len(placeholders)}__", text)
    placeholders.update({f"__TAG{i}__": m.group(0) for i, m in enumerate(re.finditer(r"<i>.*?</i>", text))})

    # {...}
    for match in re.findall(r"\{.*?\}", text):
        key = f"__BRACE{len(placeholders)}__"
        placeholders[key] = match
        text = text.replace(match, key, 1)

    # ♪
    if "♪" in text:
        key = f"__NOTE{len(placeholders)}__"
        placeholders[key] = "♪"
        text = text.replace("♪", key)

    # [ ... ]
    for match in re.findall(r"\[.*?\]", text):
        key = f"__BRACK{len(placeholders)}__"
        placeholders[key] = match
        text = text.replace(match, key, 1)

    return text, placeholders

def restore_text(text, placeholders):
    """
    Kembalikan placeholder ke bentuk asli
    """
    for key, val in placeholders.items():
        text = text.replace(key, val)
    return text

# =======================
# Translate aman
# =======================
def safe_translate(text_block, retries=9999, delay=1):
    for attempt in range(1, retries+1):
        try:
            result = translator.translate(text_block)
            if result:
                return result
        except Exception as e:
            print(f"⚠️ Error attempt {attempt}: {e}")
            time.sleep(delay + random.uniform(0.2, 0.8))
    return text_block

# =======================
# Translate SRT
# =======================
def translate_srt(file_path, batch_size=20):
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        lines = f.readlines()

    dialog_idx, dialog_lines, placeholders_all = [], [], []
    for i, line in enumerate(lines):
        if re.match(r"^\d+$", line.strip()): continue
        if "-->" in line: continue
        if line.strip() == "": continue
        dialog_idx.append(i)
        protected, ph = protect_text(line.strip())
        dialog_lines.append(protected)
        placeholders_all.append(ph)

    translated = []
    for i in tqdm(range(0, len(dialog_lines), batch_size), desc=f"Translating {os.path.basename(file_path)}"):
        batch = dialog_lines[i:i+batch_size]
        text_block = "\n".join(batch)
        translated_text = safe_translate(text_block)
        translated.extend(translated_text.split("\n"))
        time.sleep(0.5)

    for idx, new_text, ph in zip(dialog_idx, translated, placeholders_all):
        lines[idx] = restore_text(new_text, ph) + "\n"

    out_path = os.path.splitext(file_path)[0] + ".id.srt"
    with open(out_path, "w", encoding="utf-8") as f:
        f.writelines(lines)

    return out_path

# =======================
# Upload & Proses
# =======================
print("📂 Silakan upload file .srt (bisa banyak sekaligus)...")
uploaded = files.upload()

for fn in uploaded.keys():
    print(f"\n🔄 Proses: {fn}")
    out_file = translate_srt(fn, batch_size=20)
    print(f"✅ Selesai → {out_file}")
    files.download(out_file)
